# Analysis and Visuals

In [1]:
library(tidyverse)

Warning message:
"package 'tidyverse' was built under R version 3.6.3"

ERROR: Error: package or namespace load failed for 'tidyverse' in loadNamespace(j <- i[[1L]], c(lib.loc, .libPaths()), versionCheck = vI[[j]]):
 namespace 'broom' 0.5.2 is being loaded, but >= 0.7.6 is required


In [ ]:
dailyactivity <- read_csv("dailyActivity_merged.csv", 
    col_types = cols(ActivityDate = col_date(format = "%m/%d/%Y")))

In [ ]:
head(dailyactivity)

In [ ]:
hourlyactivity <- read_csv("hourlyactivity.csv")

In [ ]:
head(hourlyactivity)

In [ ]:
sleepday <- read_csv("sleepday.csv", col_types = cols(sleepday = col_date(format = "%Y-%m-%d")))

In [ ]:
head(sleepday)

In [ ]:
str(dailyactivity)

In [ ]:
str(hourlyactivity)

In [ ]:
str(sleepday)

In [ ]:
dailyactivity$day <- weekdays(dailyactivity$ActivityDate)

In [ ]:
day_check <- dailyactivity %>% 
  group_by(day) %>% 
  summarize(very_active_minutes = mean(VeryActiveMinutes), calories = mean(Calories),
            total_steps = mean(TotalSteps))

In [ ]:
print(day_check)

### Check for relationship between calories and total steps

In [ ]:
mid <- median(dailyactivity$VeryActiveMinutes)
ggplot(data=dailyactivity, aes(x=TotalSteps, y=Calories, color = VeryActiveMinutes)) +
  geom_point() + geom_smooth(method = "gam", formula = y ~s(x)) +
  scale_color_gradient2(midpoint = mid , low = "blue", mid = "green", high = "red") +
  theme_bw() +
  labs(title = "Relationship between burned calories with total steps",
       subtitle = "Comparing burned calories with total steps while highlighting very active minutes",
       caption = "figure1: Relationship between burned calories and total steps") +
  theme(plot.title = element_text(face = "bold"))

### Check for relationship between calories and sedentary minutes

In [2]:
mid <- median(dailyactivity$VeryActiveMinutes)
ggplot(data=dailyactivity, aes(x=SedentaryMinutes, y=Calories, color = VeryActiveMinutes)) +
  geom_point() + geom_smooth(method = "gam", formula = y ~s(x)) +
  scale_color_gradient2(midpoint = mid , low = "blue", mid = "green", high = "red") +
  theme_bw() +
  labs(title = "Relationship between burned calories with sedentary minutes",
       subtitle = "Comparing burned calories with sedentary minutes while highlighting very active minutes",
       caption = "figure2: Relationship between burned calories and sedentary minutes") +
  theme(plot.title = element_text(face = "bold"))

ERROR: Error in ggplot(data = dailyactivity, aes(x = SedentaryMinutes, y = Calories, : impossible de trouver la fonction "ggplot"


### Compare Total time in bed and total time asleep.

In [ ]:
ggplot(data = daily_records, aes(x = TotalMinutesAsleep, y = TotalTimeInBed)) +
  geom_point() + geom_smooth(method = "gam", formula = y ~s(x)) +
  labs(title = "Relationship between Total time in bed and Total time asleep",
       caption = "figure3: Relationship between Total time in bed and Total time asleep") +
  theme_bw() +
  theme(plot.title = element_text(face = "bold"))

In [ ]:
We go to the hourly activity data frame and add a column to separate time to dayparts(overnight, morning, midday, afternoon).

In [ ]:
T <- format(as.POSIXct(hourlyactivity$activityhour), format = "%H:%M:%S")
hourlyactivity <- hourlyactivity %>%
  mutate(DayParts = case_when(
    T < "06:00:00" & T >= "00:00:00" ~ "overnight",
    T < "10:00:00" & T >= "06:00:00" ~ "morning",
    T < "15:00:00" & T >= "10:00:00" ~ "midday",
    T < "19:00:00" & T >= "15:00:00" ~ "afternoon",
    T >= "19:00:00" ~ "evening",
    TRUE ~ "other")
    )

In [ ]:
head(hourlyactivity)

In [ ]:
hourly_review <- hourlyactivity %>% 
  group_by(DayParts) %>% 
  summarize(average_calories = mean(Calories), average_steps = mean(StepTotal))
head(hourly_review)

In [ ]:
ggplot(data = hourly_review, aes(x = DayParts, y = average_steps, fill = DayParts)) +
  geom_col() + theme_bw() +
  labs(title = "Average amount of steps walked through parts of the day",
       caption = "figure4:Average amount of steps walked through parts of the day")

In [ ]:
ggplot(data = hourly_review, aes(x = DayParts, y = average_steps, fill = DayParts)) +
  geom_col() + theme_bw() +
  labs(title = "Average amount of calories burned through parts of the day",
       caption = "figure5:Average amount of calories burned through parts of the day")

# Recommendations and key findings

### Key findings:

* From the data we have and after the analysis is done, We found that lack of activity is strongly associated with non healthy lifestyle.
* By being more active throughout the day we burn more calories which leads to healthier lifestyle.
* Having moderate amount of rest is good for being productive while having very few or too much rest leads to exhaustion.
* Most people are more active during the day.

### Recommendations:

* Notify users when sedentary time is too high.
* Make everyday target for calories to burn for each user based on her background information.
* Make an index to tell if a user had enough sleeping hours.
* Notify users to get out of bed when spending too much time in bed after waking up.
* The app should adjust targets depending on how metrics change.